# CHÚ THÍCH DATA
* customerID: ID của khách hàng
* gender: Giới tính
* SeniorCitizen: Có phải là người cao tuổi hay không ?
* Partner: Khách hàng đang có đôi tác hay không ? (đối tác)
* Dependents: Khách hàng có người bảo hộ hay không ? (bảo hộ)
* tenure: Số tháng khách hàng sử dụng dịch vụ công ty (nhiệm kỳ)
* PhoneService: KH có sử dụng dịch vụ điện thoại hay không ?
* MultipleLines: KH có nhiều đường dây hay không ? (Yes/No/Not Service)
* InternetService: Nhà cung cấp dịch vụ Internet của KH (DSL, Cáp quang, Không)
* OnlineSecurity: KH có bảo mật trực tuyến hay không ? (Có, Không, Không dịch vụ Internet)
* OnlineBackup: KH có sao lưu trực tuyến hay không (Có, Không, Không dịch vụ Internet)
* DeviceProtection: KH có bảo vệ thiết bị Internet hay không ? (Có, Không, Không dịch vụ Internet)
* TechSupport: KH có cần hỗ trợ kỹ thuật hay không (Có, Không, Không dịch vụ Internet)
* StreamingTV: KH có truyền hình trực tuyến hay không ? (Có, Không, Không dịch vụ Internet)
* StreamingMovies: KH có xem phim trực tuyến hay không ? (Có, Không, Không dịch vụ Internet)
* Contract: Thời hạn hợp đồng của khách hàng (Hàng tháng, 1 năm, 2 năm)
* PaperlessBilling: KH thanh toán mà không cần giấy tờ hhay không ? 
* PaymentMethod: Phương thức thanh toán của KH (Giao dịch điện tử, Gửi qua bưu điện, Chuyển khoản, Thẻ tín dụng)
* MonthlyCharges: Tiền phí của KH hàng tháng
* TotalCharges: Tổng phí
* Churn: KH có từ bỏ hay không ? 

In [ ]:
import pandas as pd   
import numpy as np    
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns

import time
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss, fbeta_score
from sklearn.metrics import auc, roc_curve, roc_auc_score, precision_recall_curve
from sklearn import tree

In [ ]:
df = pd.read_csv("/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

# LÀM SẠCH DỮ LIỆU

In [ ]:
df = df.dropna(how='all') 
df = df[~df.duplicated()] 
tmp = df[df.TotalCharges == ' ']
tmp

In [ ]:
# REMOVE
df['TotalCharges'] = df['TotalCharges'].replace(' ',np.nan)  
df = df.dropna(how = 'any')  
df['TotalCharges'] = df['TotalCharges'].astype(float) 

In [ ]:
print(df.Churn.value_counts())
df['Churn'].value_counts().plot(kind = 'bar').set_title('Churn')

In [ ]:
# Ta chuyển SeniorCitizen về categorical features (Yes/No) bởi vì đây không phải là 1 biến liên tục
df['SeniorCitizen'] = df['SeniorCitizen'].replace({1:'Yes',0:'No'})
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges'] 

=> Chỉ có tổng cộng 4/20 biến là numerical

In [ ]:
categorical_features = [
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'PaymentMethod',
 'PaperlessBilling',
 'Contract' ]

ROWS, COLS = 4, 4
fig, ax = plt.subplots(ROWS, COLS, figsize=(18, 20) )
row, col = 0, 0
for i, categorical_feature in enumerate(categorical_features):
    if col == COLS - 1:
        row += 1
    col = i % COLS
    df[df.Churn=='No'][categorical_feature].value_counts().plot(kind='bar', 
                width=.5, ax=ax[row, col], color='blue', alpha=0.5).set_title(categorical_feature)
    df[df.Churn=='Yes'][categorical_feature].value_counts().plot(kind='bar', 
                width=.3, ax=ax[row, col], color='orange', alpha=0.7).set_title(categorical_feature)
    plt.legend(['No Churn', 'Churn'])
    fig.subplots_adjust(hspace=0.7)

> Người cao tuổi và KH không có dịch vụ điện thoại là thiểu số trong dữ liệu 

> "Không có dịch vụ Internet" là 1 tính năng lặp lại trong 6 biêu đồ khác.

=> Ta thấy rằng tỷ lệ KH từ bỏ sẽ rơi vào KH có hợp đồng hàng tháng và thanh toán bằng Séc điện tử

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 3))
df[df.Churn == "No"][num_cols].hist(bins=35, color="blue", alpha=0.5, ax=ax)
df[df.Churn == "Yes"][num_cols].hist(bins=35, color="orange", alpha=0.7, ax=ax)
plt.legend(['No Churn', 'Churn'], shadow=True, loc=9)

Khả năng từ bỏ rơi vào những TH sau:
> Đa số ở những khách hàng mới sử dụng dịch vụ, càng về sau thì khả năng càng thấp

> Chi phí hàng tháng càng cao thì khả năng từ bỏ càng lớn 


# Feature Engineering

In [ ]:
# Chuyển MonthlyCharges sang categorical 
def monthlycharges_split(df) :   
    if df['MonthlyCharges'] <= 30 :
        return '0-30'
    elif (df['MonthlyCharges'] > 30) & (df['MonthlyCharges'] <= 70 ):
        return '30-70'
    elif (df['MonthlyCharges'] > 70) & (df['MonthlyCharges'] <= 99 ):
        return '70-99'
    elif df['MonthlyCharges'] > 99 :
        return '99plus'
df['monthlycharges_group'] = df.apply(lambda df:monthlycharges_split(df), axis = 1)

# Chuyển TotalCharges sang categorical
def totalcharges_split(df) :   
    if df['TotalCharges'] <= 2000 :
        return '0-2k'
    elif (df['TotalCharges'] > 2000) & (df['TotalCharges'] <= 4000 ):
        return '2k-4k'
    elif (df['TotalCharges'] > 4000) & (df['TotalCharges'] <= 6000) :
        return '4k-6k'
    elif df['TotalCharges'] > 6000 :
        return '6kplus'
df['totalcharges_group'] = df.apply(lambda df:totalcharges_split(df), axis = 1)

# Chuyển Tenure sang categorical 
def tenure_split(df) :   
    if df['tenure'] <= 20 :
        return '0-20'
    elif (df['tenure'] > 20) & (df['tenure'] <= 40 ):
        return '20-40'
    elif (df['tenure'] > 40) & (df['tenure'] <= 60) :
        return '40-60'
    elif df['tenure'] > 60 :
        return '60plus'
df['tenure_group'] = df.apply(lambda df:tenure_split(df), axis = 1)

In [ ]:
plt.figure(figsize = [10,5])
df[df.Churn == "No"]['monthlycharges_group'].value_counts().plot(kind = 'bar', color="blue", alpha=0.5).set_title('monthlycharges_group')
df[df.Churn == "Yes"]['monthlycharges_group'].value_counts().plot(kind = 'bar', color="orange", alpha=0.7, width=0.3)
plt.legend(['No Churn', 'Churn'], shadow=True, loc=1)

In [ ]:
plt.figure(figsize = [10,5])
df[df.Churn == "No"]['totalcharges_group'].value_counts().plot(kind = 'bar', color="blue", alpha=0.5).set_title('totalcharges_group')
df[df.Churn == "Yes"]['totalcharges_group'].value_counts().plot(kind = 'bar', color="orange", alpha=0.7, width=0.3)
plt.legend(['No Churn', 'Churn'], shadow=True, loc=1)

In [ ]:
plt.figure(figsize = [10,5])
df[df.Churn == "No"]['tenure_group'].value_counts().plot(kind = 'bar', color="blue", alpha=0.5).set_title('tenure_group')
df[df.Churn == "Yes"]['tenure_group'].value_counts().plot(kind = 'bar', color="orange", alpha=0.7, width=0.3)
plt.legend(['No Churn', 'Churn'], shadow=True, loc=1)

In [ ]:
df.to_csv('df.csv', index=False)

In [ ]:
df = pd.read_csv('df.csv')

In [ ]:
# Xử lý dữ liệu

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

Id_col     = ['customerID']
target_col = ['Churn']
# Biến số
cat_cols   = df.nunique()[df.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
# Biến phân loại
num_cols   = [x for x in df.columns if x not in cat_cols + target_col + Id_col]
# Biến nhị phân
bin_cols   = df.nunique()[df.nunique() == 2].keys().tolist()
# Các biến > 2 giá trị
multi_cols = [i for i in cat_cols if i not in bin_cols]

# Label encoding Binary
le = LabelEncoder()
for i in bin_cols :
    df[i] = le.fit_transform(df[i])
    
df = pd.get_dummies(data = df, columns = multi_cols)

#Scaling biên số
std = StandardScaler()
scaled = std.fit_transform(df[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)
df1 = df.drop(columns = num_cols, axis = 1)
df1 = df1.merge(scaled, left_index=True, right_index=True, how = "left")

In [ ]:
# Loại bỏ cột customerID
df1 = df1.drop('customerID', axis=1)

# Nhiều cột đều có (no internet service). Chỉ giữ lại 1
df1 = df1.drop(columns=['OnlineSecurity_No internet service', 'OnlineBackup_No internet service', 
                        'DeviceProtection_No internet service', 'TechSupport_No internet service', 
                        'StreamingTV_No internet service', 'StreamingMovies_No internet service'], axis=1)


In [ ]:
df1.to_csv('df1.csv', index=False)

# Feature Selection

In [ ]:
df1 = pd.read_csv('df1.csv')
df1

In [ ]:
X, y = df1.drop('Churn',axis=1), df1[['Churn']]

In [ ]:
from sklearn import tree

def model_report(model_name, model):
    print('\nSearch for OPTIMAL THRESHOLD, vary from 0.0001 to 0.9999, fit/predict on train/test data')
    model.fit(X_train, y_train)
    optimal_th = 0.5   
    
    for i in range(0,3):
        score_list = []
        print('\nLooping decimal place', i+1) 
        th_list = [np.linspace(optimal_th-0.4999, optimal_th+0.4999, 11), 
                 
                 np.linspace(optimal_th-0.1, optimal_th+0.1, 21), 
                 np.linspace(optimal_th-0.01, optimal_th+0.01, 21)]
        for th in th_list[i]:
            y_pred = (model.predict_proba(X_test)[:,1] >= th)
            f1scor = f1_score(y_test, y_pred)
            score_list.append(f1scor)
            print('{:.3f}->{:.4f}'.format(th, f1scor), end=',  ')  
        optimal_th = float(th_list[i][score_list.index(max(score_list))])

    print('optimal F1 score = {:.4f}'.format(max(score_list)))
    print('optimal threshold = {:.3f}'.format(optimal_th))

    print(model_name, 'accuracy score is')
    print('Training: {:.2f}%'.format(100*model.score(X_train, y_train))) 
    print('Test set: {:.2f}%'.format(100*model.score(X_test, y_test)))   

    y_pred = (model.predict_proba(X_test)[:,1] >= 0.25)
    print('\nAdjust threshold to 0.25:')
    print('Precision: {:.4f},   Recall: {:.4f},   F1 Score: {:.4f}'.format(
        precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred)))
    print(model_name, 'confusion matrix: \n', confusion_matrix(y_test, y_pred))

    y_pred = model.predict(X_test)
    print('\nDefault threshold of 0.50:')
    print('Precision: {:.4f},   Recall: {:.4f},   F1 Score: {:.4f}'.format(
        precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred)))
    print(model_name, 'confusion matrix: \n', confusion_matrix(y_test, y_pred))

    y_pred = (model.predict_proba(X_test)[:,1] >= 0.75)
    print('\nAdjust threshold to 0.75:')
    print('Precision: {:.4f},   Recall: {:.4f},   F1 Score: {:.4f}'.format(
        precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred)))
    print(model_name, 'confusion matrix: \n', confusion_matrix(y_test, y_pred))

    y_pred = (model.predict_proba(X_test)[:,1] >= optimal_th)
    print('\nOptimal threshold {:.3f}'.format(optimal_th))
    print('Precision: {:.4f},   Recall: {:.4f},   F1 Score: {:.4f}'.format(
        precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred)))
    print(model_name, 'confusion matrix: \n', confusion_matrix(y_test, y_pred))
    
    global model_f1, model_auc, model_ll, model_roc_auc
    model_f1 = f1_score(y_test, y_pred)

    y_pred = model.predict_proba(X_test)
    model_ll = log_loss(y_test, y_pred)
    print(model_name, 'Log-loss: {:.4f}'.format(model_ll))
    y_pred = model.predict(X_test)
    model_roc_auc = roc_auc_score(y_test, y_pred)
    print(model_name, 'roc_auc_score: {:.4f}'.format(model_roc_auc)) 
    y_pred = model.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    model_auc = auc(fpr, tpr)
    print(model_name, 'AUC: {:.4f}'.format(model_auc))

    plt.figure(figsize = [6,6])
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % model_auc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.savefig('roc_auc_score')
    #tree.plot_tree(model)
    plt.show()
  
    return

# Model Selection

In [ ]:
# chia 80:20
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=71)
print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)

# DECISION TREE CLASSIFIER

In [ ]:
from xgboost import plot_tree

In [ ]:
print('\n"""""" Cây Phân loại """"""')

print('\nTìm tối ưu `, thay đổi từ 3 -> 20, using KFold(5) Cross Validation trên tập train')
kf = KFold(n_splits=5, random_state=21, shuffle=True) 
d_scores = []
for d in range(3, 20):
    decisiontree = DecisionTreeClassifier(max_depth=d)
    cvs = cross_val_score(decisiontree, X_train, y_train, cv=kf, scoring='f1').mean()
    d_scores.append(cvs)
    print('{:.4f}'.format(cvs), end=", ")
print('F1 scpre được chọn = {:.4f}'.format(max(d_scores))) 
optimal_d = d_scores.index(max(d_scores)) + 2   # Chỉ số = 0 tức là dept  = 3
print('Độ sâu tối ưu =', optimal_d)

time1 = time.time()
decisiontree = DecisionTreeClassifier(max_depth=optimal_d, criterion = "entropy")
model_report('DecisionTreeClassifier', decisiontree)


**K-FOLD Cross Validation**

KFold(n_splits=5, random_state=21, shuffle=True)
> n_splits=5: Number of fold

> shuffle: Whether shuffle data before split into each fold
 
> random_state: Have effect when set shuffle = True, mỗi lần random cho mỗi fold sẽ lấy cách nhau chỉ số là 21 nhằm đảm bảo tính ngẫu nhiên
 



In [ ]:
df1.head

In [ ]:
X_train.columns
